## Deliverable 1. Retrieve Weather Data

In [1]:
# Use the citipy module to determine city based on latitude and longitude
from citipy import citipy

# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
print(f'There are {len(cities)} cities')

There are 763 cities


In [5]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=ee23d400752435fdd0ad6fd586b19ba1


In [7]:
# Create counters.
record_count = 1
set_count = 1
city_data = []

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather['weather'][0]['description']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        pass

In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,Kodiak,57.7900,-152.4072,30.60,93,100,11.10,US,snow,2022-01-09 06:47:54
1,Kannur,11.8675,75.3576,85.15,58,8,7.78,IN,clear sky,2022-01-09 06:47:39
2,Port Elizabeth,-33.9180,25.5701,78.10,83,75,17.27,ZA,broken clouds,2022-01-09 06:43:13
3,Kazerun,29.6195,51.6541,58.41,29,0,1.05,IR,clear sky,2022-01-09 06:47:55
4,Maarianhamina,60.0973,19.9348,31.86,100,100,5.75,AX,fog,2022-01-09 06:47:55
5,Butaritari,3.0707,172.7902,79.79,84,100,17.22,KI,moderate rain,2022-01-09 06:47:56
6,Coihaique,-45.5752,-72.0662,55.35,67,75,2.30,CL,broken clouds,2022-01-09 06:47:56
7,Alta Floresta,-9.8756,-56.0861,70.81,98,100,5.53,BR,overcast clouds,2022-01-09 06:47:57
8,Bluff,-46.6000,168.3333,61.81,96,100,6.20,NZ,overcast clouds,2022-01-09 06:47:57
9,Cape Town,-33.9258,18.4232,78.10,59,0,14.97,ZA,clear sky,2022-01-09 06:43:13


In [9]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")